In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost

In [116]:
data = pd.read_excel("../data/农业碳排放数据.xlsx",index_col=0,sheet_name=None)

In [123]:
def fill_use_xgboost(data):
    if data.isna().sum().sum()>0:
        data=data.copy()


        imp = IterativeImputer(
            estimator=xgboost.XGBRegressor(
                n_estimators=10,
                random_state=1
                # tree_method='gpu_hist',
            ),
            missing_values=np.nan,
            max_iter=5,
            initial_strategy='mean',
            imputation_order='ascending',
            verbose=2,
            random_state=1
        )

        data[:] = imp.fit_transform(data)
        return data
    else:
        return data

In [124]:
def fill_na(data,save_file):
    with pd.ExcelWriter(save_file) as writer:
        for k,v in data.items():
            fill_use_xgboost(v).to_excel(writer,sheet_name =k)

fill_na(data,r"../碳排放总量_fillna.xlsx")

[IterativeImputer] Completing matrix with shape (21, 21)
[IterativeImputer] Ending imputation round 1/5, elapsed time 0.33
[IterativeImputer] Change: 10111.429964192708, scaled tolerance: 60858.100000000006 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (21, 21)
[IterativeImputer] Ending imputation round 1/5, elapsed time 0.32
[IterativeImputer] Change: 518932.75265896635, scaled tolerance: 5850.0 
[IterativeImputer] Ending imputation round 2/5, elapsed time 0.67
[IterativeImputer] Change: 1984.931640625, scaled tolerance: 5850.0 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (21, 21)
[IterativeImputer] Ending imputation round 1/5, elapsed time 0.33
[IterativeImputer] Change: 610227.5948881272, scaled tolerance: 11390.0 
[IterativeImputer] Ending imputation round 2/5, elapsed time 0.68
[IterativeImputer] Change: 14978.359497070312, scaled tolerance: 11390.0 
[IterativeImputer] 

In [125]:
def data_pro(series):
    s1 = series.shift(1)
    s1.iloc[0] = series.iloc[0]
    return (s1+series)/2

def cal_grow(series):
    s1 = series.shift(1)
    s1.iloc[0] = series.iloc[0]
    out = ((series-s1)/s1)*100
    return out

In [126]:
def cal_C_out(data):
    C_list = []
    C1_list = []
    C1_occupy = []
    C2_list = []
    C2_occupy = []
    C3_list = []
    C3_occupy = []
    C4_list = []
    C4_occupy = []
    
    # 处理牛羊数据
    data['牛存栏数量（头）'] = data_pro(data['牛存栏数量（头）'])
    data['羊存栏数量（只'] = data_pro(data['羊存栏数量（只）'])
    
    for item in data.iterrows():
        line = item[1]
        
        
        '''
        # 化肥
        #农药
        #农膜
        #农业灌溉
        '''
        c1 = (line[0] * 1000 * 3.284) \
            + (line[1] * 1000 * 18.092) \
                + (line[2] * 1000 * 18.993) \
                    + (line[3] * (266.48/12)*44)
            
            
        '''
        #棉花播种面积
        #蔬菜播种面积
        #水稻播种面积
        # + (line[7] * 2.05)  # 冬小麦
        #大豆
        #玉米
        ''' 
        c2 =  (line[4] * 0.4804*256)\
            + (line[5] * 4.21*256) \
                + (line[6] * (210*28 + 0.24*256)) \
                    + (line[7] * 0.77*256)  \
                        + (line[8] * 2.532*256)
            
        '''
        # 猪
        #家禽
        #兔
        #牛
        #羊
        '''
        c3 = (((line[9] * 200)/365) * (1*28 + 3.5*28 + 0.53*256)) \
            +(((line[10] *55)/365) * (0.02*28 + 0.02*256)) \
                + (((line[11] *105)/365) * (0.254*28 + 0.08*28 + 0.02*256)) \
                    + ((line[12] * (47.8*28 + 1*28 + 1.39*256)))\
                        + ((line[13] * (5*28+0.16*28 + 0.33*256)))

        # break
        
        '''
        #水稻
        #小麦  + line[15] *1000 * 1.1 * 23.8 * 0.93 * (60 + 1460) \
        #玉米
        #大豆
        #薯类
        #油料
        #棉花
        '''
        c4 =  line[14] *1000 * 0.9 * 26.8 * 0.93 * (3.2*28 + 1460) \
        + line[15] *1000 * 1.2 * 17.2 * 0.92 * (4.4*28 + 1350) \
        + line[16] *1000 * 1.6 * 23.2 * 0.68 * (3.9*28 + 1445) \
        + line[17] *1000 * 0.5 * 23.2 * 0.68 * (3.9*28 + 1445) \
        + line[18] *1000 * 1.5 * 23.2 * 0.82 * (3.5*28 + 1445) \
        + line[19] *1000 * 9.2 * 25.1 * 0.80 * (3.5*28 + 1445) 
        c4 = c4/100000
        c1 = c1/1000
        c2 = c2/1000
        c3 = c3/1000
        c4 = c4/1000
        C = c1+c2+c3+c4
    
        # C_list.append((c1/1000,c2/1000,c3/1000,c4/1000,(C/1000)))
        
        C1_list.append(c1/10000)
        C1_occupy.append(c1/C*100)
        
        C2_list.append(c2/10000)
        C2_occupy.append(c2/C*100)
        
        C3_list.append(c3/10000)
        C3_occupy.append(c3/C*100)
        
        C4_list.append(c4/10000)
        C4_occupy.append(c4/C*100)
        
        C_list.append(C/10000)
        
    df = pd.DataFrame(C1_list,index=data.index,columns=["农资投入（×10^4 t）"])
    df["C1占比（%）"] = C1_occupy
    
    df["农田土壤利用（×10^4 t）"] = C2_list
    df["C2占比（%）"] = C2_occupy
    
    df["畜禽养殖（×10^4 t）"] = C3_list
    df["C3占比（%）"] = C3_occupy
    
    df["秸秆焚烧（×10^4 t）"] = C4_list
    df["C4占比（%）"] = C4_occupy
    
    df["碳排放总量（×10^4 t）"] = C_list
    
    df["同比变化"] = cal_grow(df["碳排放总量（×10^4 t）"])
    
    return df


In [128]:
data = pd.read_excel("../data/江西省碳排放数据_fillna.xlsx",index_col=0,sheet_name=None)

In [129]:
def cal_all_area_C(data,save_file):
    with pd.ExcelWriter(save_file) as writer:
        for k,v in data.items():
            cal_C_out(v).to_excel(writer,sheet_name =k)

cal_all_area_C(data,r"../碳排放总量.xlsx")

In [36]:
def cal_C_out_intensity():
    data_C = pd.read_excel("../data/碳排放总量.xlsx",sheet_name=None,index_col=0)
    data_Arg_anim = pd.read_excel('../data/农牧业生产总值.xlsx',index_col=0)
    C_intensity = []
    for k,v in data_C.items():
        data_c_all = v["碳排放总量（×10^4 t）"]
        C_intensity.append(data_c_all.values/data_Arg_anim[k].values)
    return pd.DataFrame(np.array(C_intensity).T,index=data_Arg_anim.index,columns=list(data_C))
    
cal_C_out_intensity().to_excel("../data/碳排放强度.xlsx")

In [40]:
def cal_cv(C=True):
    if C:
        out = []
        data_C = pd.read_excel("../data/碳排放总量.xlsx",sheet_name=None,index_col=0)
        for k,v in data_C.items():
            if k !="江西省":
                out.append(v["碳排放总量（×10^4 t）"].values)
        data = np.array(out).T
        cv = data.std(axis=1)/data.mean(axis=1)*100
        data = pd.DataFrame(data,index=v.index,columns=list(data_C)[1:])
        data["cv（%）"] = cv
        return data
    else:
        data_C_intensity = pd.read_excel("../data/碳排放强度.xlsx",index_col=0).iloc[:,1:]
        cv = data_C_intensity.std(axis=1)/data_C_intensity.mean(axis=1)*100
        data_C_intensity["cv（%）"] = cv
        return data_C_intensity

cal_cv(C=True).to_excel("../data/碳排放总量_CV.xlsx")


In [39]:
cal_cv(C=False).to_excel("../data/碳排放强度_CV.xlsx")

In [35]:
# 计算各市碳排放总量cv
def get_C_all():
    out = []
    data_C = pd.read_excel("../data/碳排放总量.xlsx",sheet_name=None,index_col=0)
    for k,v in data_C.items():
        out.append(v["碳排放总量（×10^4 t）"].values)
    data = pd.DataFrame(np.array(out).T,index=v.index,columns=list(data_C))
    return data
    
get_C_all().to_excel("../data/江西省及各市碳排放总量.xlsx")